# Линейная алгебра в NumPy

Очень часто различные научные вычисления и алгоритмы связаны с операциями линейной алгебры: перемножением матриц и векторов, вычислением обратных матриц и собственных значений матриц, осуществлением различных разложений матриц и т.д. Все эти операции играют важную роль, например, в методах оптимизации или в машинном обучении. Именно поэтому основные алгоритмы линейной алгебры были реализованы в виде специального подмодуля библиотеки NumPy - `linalg` (сокращение от *linear algebra*). В сегодняшнем семинаре мы посмотрим на некоторый функционал этого модуля.

**Необходимые импорты:**

In [1]:
import numpy as np

Для воспроизводимости результатов, зафиксируем random-seed:

In [2]:
np.random.seed(42)

## Объединение и разбиение массивов

Прежде чем перейти к изучению подмодуля `linalg`, рассмотрим пару полезных функций для объединения и разделения массивов. Эти функции могут оказаться полезными по ряду причин. В качестве яркого примера таких причин можно назвать оптимизацию вычислений. В случае, когда нам необходимо применить некоторую операцию к нескольким массивам, мы можем объединить исходные массивы в один большой массив, чтобы векторизовать вычисления и избежать использования медлительного цикла `for`. После применения желаемой операции к полученному массиву мы можем разбить итоговый массив на несколько массивов, которые будут являться результатом вычислений, примененных к соответствующим входным массивам. Также операции объединения массивов могут быть полезны непосредственно при решении задач линейной алгебры. Например, решая систему линейных уравнений, с помощью функций объединения массивов, вы сможете составить расширенную матрицу системы.

### append

В отличие от списков в Python, массивы в NumPy не определяют методы для добавления новых элементов в конец. Для того, чтобы добавить новые данные в конец массива NumPy придется использовать отдельную функций - `np.append`. В качестве параметров `np.append` принимает массив, в конец которого необходимо дописать данные, массив, который используется в качестве источника данных для дописывания, а также необязательный параметр `axis`, с которым мы познакомились на прошлом семинаре. Рассмотрим примеры использования `np.append`:

In [7]:
array_1d = np.random.randint(-10, 10, size=2)
array_2d = np.random.randint(-10, 10, size=(2, 2))

print(
    f"array_1d:\n{array_1d}",
    f"array_2d:\n{array_2d}",
    f"1D: append 1D:\n{np.append(array_1d, array_1d)}",
    f"1D: append 2D:\n{np.append(array_1d, array_2d)}",
    f"2D: append rows:\n{np.append(array_2d, array_2d, axis=0)}",
    f"2D: append cols:\n{np.append(array_2d, array_1d[..., np.newaxis], axis=1)}",
    sep="\n\n",
)

array_1d:
[1 6]

array_2d:
[[-1  5]
 [ 4  4]]

1D: append 1D:
[1 6 1 6]

1D: append 2D:
[ 1  6 -1  5  4  4]

2D: append rows:
[[-1  5]
 [ 4  4]
 [-1  5]
 [ 4  4]]

2D: append cols:
[[-1  5  1]
 [ 4  4  6]]


На приведенных примерах можно пронаблюдать особенности функции `np.append`. Первая особенность, составляющая ключевое отличие от аналогичного метода списков в "чистом" Python, заключается в том, что `np.append` не применяется на месте. Т.е. в результате выполнения функции входные данные не модифицируются, а результат выполнения - новый объект в памяти.

In [5]:
array_appended = np.append(array_1d, array_1d)

print(
    f"array_1d:\n{array_1d}",
    f"array_appended:\n{array_appended}",
    f"is the same: {array_1d is array_appended}",
    sep="\n\n",
)

array_1d:
[8 0]

array_appended:
[8 0 8 0]

is the same: False


Следующая особенность состоит в возможности использования многомерных массивов и в качестве источника данных, и в качестве расширяемого массива. В случае если расширяемый массив является одномерным массивом, в результате выполнения функции все данные из второго массива будут дописаны в конец расширяемого массива. В этом случае поведение `np.append` можно сравнить с поведением метода списков `extend`.

Если же переданные массивы многомерные, их размерности должны быть равными. Т.е. значения атрибутов `ndim` должны быть равны. Иначе `np.append` возбудит исключение. В случае использования многомерных массивов вы также получаете возможность использовать параметра `axis`, чтобы выбирать измерение, вдоль которого будут добавлены данные. По умолчанию массивы будут вытянуты в одномерный массив и объединены.

In [8]:
print(np.append(array_2d, array_2d))

[-1  5  4  4 -1  5  4  4]


### concatenate

Следующая функция, позволяющая объединять массивы - `np.concatenate`. В отличие от `np.append`, `np.concatenate` позволяет объединить произвольное число массивов и при это явно задать порядок их объединения. В случае многомерных массивов размерности объединяемых массивов должны быть равны, иначе функция возбудит исключение. `np.concatenate`, также как и `np.append`, не модифицирует исходные данные. Однако в отличие от `np.append` `np.concatenate` не всегда создает новый объект в памяти: при передаче параметра `out` `np.concatenate` будет записывать результат в уже существующий в памяти объект.

In [10]:
array_1d = np.random.randint(-10, 10, size=2)
array_2d = np.random.randint(-10, 10, size=(2, 2))

print(
    f"array_1d:\n{array_1d}",
    f"array_2d:\n{array_2d}",
    f"concatenate no axis:\n"
    f"{np.concatenate((array_1d, array_2d), axis=None)}",
    f"concatenate with axis:\n"
    f"{np.concatenate((array_2d, array_1d[np.newaxis, ...]), axis=0)}",
    sep="\n\n",
)

array_1d:
[-4 -2]

array_2d:
[[-4  7]
 [-7  3]]

concatenate no axis:
[-4 -2 -4  7 -7  3]

concatenate with axis:
[[-4  7]
 [-7  3]
 [-4 -2]]


Как вы видите, с помощью добавления новых размерностей и указания аргумента `axis` мы можем регулировать, как именно значения массивов будут объединяться, и какой формы будет результирующий массив. Однако при работе с матрицами нам не нужен такой уровень гибкости. В большинстве случаев нам будет необходимо или приписывать столбцы (`axis=1`), или приписывать строки (`axis=0`). Именно для таких случаев в NumPy были добавлены специализированные функции.

### hstack и vstack

Пара функций `np.hstack` (сокращение от *horizontal stacking*) и `np.vstack` (аналогично, *vertical stacking*) позволяют дописывать строки и столбцы соответственно.

In [3]:
array_1d = np.random.randint(-10, 10, size=2)
array_2d = np.random.randint(-10, 10, size=(2, 2))

print(
    f"array_1d:\n{array_1d}",
    f"array_2d:\n{array_2d}",
    "hstack 1d to 2d:\n"
    f"{np.hstack((array_2d, array_1d[..., np.newaxis]))}",
    "hstack 2d to 1d:\n"
    f"{np.hstack((array_1d[..., np.newaxis], array_2d))}",
    "vstack 1d to 2d:\n"
    f"{np.vstack((array_2d, array_1d))}",
    "vstack 2d to 1d:\n"
    f"{np.vstack((array_1d, array_2d))}",
    sep="\n\n",
)

array_1d:
[8 0]

array_2d:
[[ 0 -7]
 [-3 -8]]

hstack 1d to 2d:
[[ 0 -7  8]
 [-3 -8  0]]

hstack 2d to 1d:
[[ 8  0 -7]
 [ 0 -3 -8]]

vstack 1d to 2d:
[[ 0 -7]
 [-3 -8]
 [ 8  0]]

vstack 2d to 1d:
[[ 8  0]
 [ 0 -7]
 [-3 -8]]


### split

Очевидно, если есть возможность слияние массивов в один, также существует возможность разделения одного массива на несколько массивов. Начнем с первого антипода - антипод функции `np.concatenate` - `np.split`. С помощью данной функции вы можете разбить переданный массив на несколько подмассивов по заданному правилу вдоль заданного направления.

In [12]:
def convert_npsplit_to_string(
    parts: list[np.ndarray],
) -> str:
    return ",\n".join(
        [str(part) for part in parts]
    )

In [13]:
array = np.random.randint(-10, 10, size=(6, 4))

print(
    f"array:\n{array}",
    "split-use-int-rule:\n"
    f"{convert_npsplit_to_string(np.split(array, 3))}",
    "split-use-slice-rule:\n"
    f"{convert_npsplit_to_string(np.split(array, [1, 4]))}",
    "split-use-axis:\n"
    f"{convert_npsplit_to_string(np.split(array, 2, axis=1))}",
    sep="\n\n",
)

array:
[[ -7   3   5   4]
 [ -3   3  -3   5]
 [  2   7   4   2]
 [ -2   4   2 -10]
 [ -4  -2 -10   1]
 [ -3   0   8   6]]

split-use-int-rule:
[[-7  3  5  4]
 [-3  3 -3  5]],
[[  2   7   4   2]
 [ -2   4   2 -10]],
[[ -4  -2 -10   1]
 [ -3   0   8   6]]

split-use-slice-rule:
[[-7  3  5  4]],
[[ -3   3  -3   5]
 [  2   7   4   2]
 [ -2   4   2 -10]],
[[ -4  -2 -10   1]
 [ -3   0   8   6]]

split-use-axis:
[[-7  3]
 [-3  3]
 [ 2  7]
 [-2  4]
 [-4 -2]
 [-3  0]],
[[  5   4]
 [ -3   5]
 [  4   2]
 [  2 -10]
 [-10   1]
 [  8   6]]


### hsplit и vsplit

Также аналоги-разделители определены и для функций `np.hstack` и `np.vstack` - `np.hsplit` и `np.vsplit`. Обе эти функции аналогичны функции `np.split` с фиксированным значением аргумента `axis`: `axis=0` для `vsplit`, `axis=1` для `hsplit`.

In [14]:
array = np.random.randint(-10, 10, size=(6, 4))

print(
    f"array:\n{array}",
    "vsplit:\n"
    f"{convert_npsplit_to_string(np.vsplit(array, 3))}",
    "hsplit:\n"
    f"{convert_npsplit_to_string(np.hsplit(array, [1, 3]))}",
    sep="\n\n",
)

array:
[[ -3  -8  -8 -10]
 [ -6  -1  -4  -2]
 [ -4  -2  -3   1]
 [ -9 -10   5  -6]
 [ -8   1  -3  -8]
 [-10  -8  -6   4]]

vsplit:
[[ -3  -8  -8 -10]
 [ -6  -1  -4  -2]],
[[ -4  -2  -3   1]
 [ -9 -10   5  -6]],
[[ -8   1  -3  -8]
 [-10  -8  -6   4]]

hsplit:
[[ -3]
 [ -6]
 [ -4]
 [ -9]
 [ -8]
 [-10]],
[[ -8  -8]
 [ -1  -4]
 [ -2  -3]
 [-10   5]
 [  1  -3]
 [ -8  -6]],
[[-10]
 [ -2]
 [  1]
 [ -6]
 [ -8]
 [  4]]


## Операции с матрицами

Начнем изучение функционала NumPy для работы с операциями линейной алгебры с операций, доступных без подключения подмодуля `linalg`. А именно - рассмотрим все возможные произведения.

### Оператор @ и скалярные произведения

Начиная с версии 3.5 в Python был добавлен новый оператор - `@`, который в контексте массивов NumPy имеет смысл обычного произведения матриц:

In [15]:
vector1 = np.random.randint(-10, 10, size=3)
vector2 = np.random.randint(-10, 10, size=3)

matrix1 = np.random.randint(-10, 10, size=(4, 3))
matrix2 = np.random.randint(-10, 10, size=(3, 4))

print(
    f"vector1:\n{vector1}",
    f"vector2:\n{vector2}",
    f"matrix1:\n{matrix1}",
    f"matrix2:\n{matrix2}",
    sep="\n\n",
)

vector1:
[  3  -8 -10]

vector2:
[-6  3 -4]

matrix1:
[[-2  4  4]
 [-1  2  8]
 [-4  6  9]
 [-7 -6 -4]]

matrix2:
[[ 2  4  0 -7]
 [ 2 -4  8 -9]
 [-1  2 -5  1]]


In [16]:
print(
    f"vector @ vector:\n{vector1 @ vector2}",
    f"vector1 @ matrix2:\n{vector1 @ matrix2}",
    f"matrix1 @ vector2:\n{matrix1 @ vector2}",
    f"matrix @ matrix:\n{matrix1 @ matrix2}",
    sep="\n\n",
)

vector @ vector:
-2

vector1 @ matrix2:
[  0  24 -14  41]

matrix1 @ vector2:
[  8 -20   6  40]

matrix @ matrix:
[[  0 -16  12 -18]
 [ -6   4 -24  -3]
 [ -5 -22   3 -17]
 [-22 -12 -28  99]]


Из данных примеров видно, что одномерный массив задает как вектор-строки, так и вектор-столбцы, а оператор `@` относится к произведению одномерных массивов, как к произведениям вектора-строки на вектор-столбец, т.е. как к обычному скалярному произведению векторов. Но как быть, если мы хотим получить матрицу при выполнении операции `@`? Т.е., как быть, если нам необходимо осуществить произведение вектор-столбца на вектор-строку? Для решения данной проблемы существуют два решения: манипуляция с размерностями и функция `np.outer`. Ниже рассмотрены оба подхода.

In [18]:
vector1 = np.random.randint(-10, 10, size=3)
vector2 = np.random.randint(-10, 10, size=3)

print(
    f"vector1:\n{vector1}",
    f"vector2:\n{vector2}",
    "shape manipulation:\n"
    f"{vector1[..., np.newaxis] @ vector2[np.newaxis, ...]}",
    f"np.outer:\n{np.outer(vector1, vector2)}",
    sep="\n\n",
)

vector1:
[ 9  2 -2]

vector2:
[-8 -4 -5]

shape manipulation:
[[-72 -36 -45]
 [-16  -8 -10]
 [ 16   8  10]]

np.outer:
[[-72 -36 -45]
 [-16  -8 -10]
 [ 16   8  10]]


Как и в случае с векторизованными операциями, оператор `@` имеет функциональную форму `np.matmul`, с помощью которой можно указать буфер для записи результата.

### Векторное произведение

Векторное произведение не имеет операторной формы записи, но имеет функциональную реализацию в виде функции `np.cross`. Функция принимает на вход массивы различных размерностей и предоставляет возможность указать измерения массивов, вдоль которых расположены векторы для выполнения вычислений.

In [19]:
vector = np.random.randint(-10, 10, size=3)
pack_of_vectors = np.random.randint(-10, 10, size=(3, 3))

print(
    f"vector:\n{vector}",
    f"pack of vectors:\n{pack_of_vectors}",
    f"cross product:\n{np.cross(vector, pack_of_vectors, axis=-1)}",
    sep="\n\n",
)

vector:
[-3 -2 -6]

pack of vectors:
[[-10   8  -1]
 [  1   4  -2]
 [  9   6   6]]

cross product:
[[ 50  57 -44]
 [ 28 -12 -10]
 [ 24 -36   0]]


### Задание 1. Смешанное произведение

Итак, в NumPy есть скалярное и векторное произведение. Но, к сожалению, нет смешанного произведения векторов. Чтобы исправить это упущение, реализуем функцию для вычисления смешанного произведения, заложив туда возможность вычислять смешанное произведение для целого батча векторов. Итак, необходимо реализовать функцию, на вход которой подается или три вектора, или три матрицы. В случае векторов результат выполнения функции - число - смешанное произведение векторов. В случае матриц результат - n чисел - смешанные произведения. n - число векторов в матрицах.

В случае если матрицы имеют разные размеры, необходимо возбудить исключение `ShapeMismatchError`. В случае, если хотя бы одна из переданных матриц имеет размерность, большую двух, необходимо возбудить исключение `ShapeMismatchError`.

Замечание, считаем, что вектора в матрице храняться вдоль измерения axis=1.

**Напоминание**

Смешнным прозведением векторов $(\vec{a}, \vec{b}, \vec{c})$ называют скалярное прозведение векторов $(\vec{a}, \vec{h})$, где вектор $\vec{h} = \vec{b} \times \vec{c}$ - векторное произведение векторов $\vec{b}, \vec{c}$.

**Необходимые импорты**:

In [11]:
from typing import Union
from numbers import Real

**Программная реализация**:

In [12]:
class ShapeMismatchError(Exception):
    pass

In [13]:
def get_mixed_product(
    vec1: np.ndarray,
    vec2: np.ndarray,
    vec3: np.ndarray,
) -> Union[np.ndarray, Real]:
    # ваш код
    
    return 0

**Тесты**:

In [14]:
vec1 = np.arange(3)
vec2 = np.array([1, 0, 0])
vec3 = np.array([0, 1, 0])

result = get_mixed_product(vec1, vec2, vec3)
assert result == 2

result = get_mixed_product(
    np.repeat(vec1[np.newaxis, ...], 3, axis=0),
    np.repeat(vec2[np.newaxis, ...], 3, axis=0),
    np.repeat(vec3[np.newaxis, ...], 3, axis=0),
)
assert np.all(result == np.full_like(vec1, fill_value=2))

try:
    get_mixed_product(
        np.repeat(vec1[np.newaxis, ...], 3, axis=0),
        vec2,
        vec3,
    )

except ShapeMismatchError:
    pass

else:
    assert False, "exception expected"

try:
    get_mixed_product(
        vec1[np.newaxis, np.newaxis, ...],
        vec2,
        vec3,
    )

except ShapeMismatchError:
    pass

else:
    assert False, "exception expected"

### Вычисление основных чисел матриц

**Норма**

Во многих задачах полезным бывает вычислить норму вектора, например, для последующей нормировки этого вектора. Для этой цели стоит использовать функцию `np.linalg.norm`, которая обладает очень широким функционалом и позволяет вычислять как нормы векторов, так и нормы матриц. Причем, с помощью аргумента `ord` вы можете выбрать, какую именно норму следует вычислить. По умолчанию будет вычислена знакомая вам со школы l2-норма для векторов. В общем же случае эта норма носит название Евклидовой нормы или нормы Фробениуса. Также у этой функции есть аргумент `axis`, благодаря которому вы получаете возможность вычисления нормы вдоль указанных измерений массива. 

In [20]:
matrix = np.random.randint(-10, 10, size=(3, 3))

print(
    f"matrix:\n{matrix}",
    f"matrix norm:\n{np.linalg.norm(matrix)}",
    f"vectors' norms\n{np.linalg.norm(matrix, axis=-1)}",
    "custom vectors' norm:\n"
    f"{np.sqrt(np.sum(matrix ** 2, axis=-1))}",
    sep="\n\n",
)

matrix:
[[ 9  1 -4]
 [-9 -8  6]
 [-6  6  6]]

matrix norm:
19.672315572906

vectors' norms
[ 9.89949494 13.45362405 10.39230485]

custom vectors' norm:
[ 9.89949494 13.45362405 10.39230485]


**Ранг**

Для выполнения ряда операций требуется определение ранга матрицы. Например, определение ранга матрицы может быть необходимым шагом перед обращением матрицы во время решения матричных уравнений. В NumPy определение ранга матрицы осуществляется с помощью функции `np.linalg.matrix_rank`, в основе которой лежит алгоритм сингулярного разложения матриц.

In [21]:
matrix_full_rank = np.diag(np.random.randint(1, 5, size=4))
matrix_not_full_rank = matrix_full_rank.copy()
matrix_not_full_rank[1, 1] = 0

print(
    f"full-rank matrix:\n{matrix_full_rank}",
    f"not full-rank matrix:\n{matrix_not_full_rank}",
    "full-rank matrix rank:\n"
    f"{np.linalg.matrix_rank(matrix_full_rank)}",
    "not full-rank matrix rank:\n"
    f"{np.linalg.matrix_rank(matrix_not_full_rank)}",
    sep="\n\n",
)

full-rank matrix:
[[3 0 0 0]
 [0 1 0 0]
 [0 0 2 0]
 [0 0 0 2]]

not full-rank matrix:
[[3 0 0 0]
 [0 0 0 0]
 [0 0 2 0]
 [0 0 0 2]]

full-rank matrix rank:
4

not full-rank matrix rank:
3


**Детерминант**

Детерминант матрицы является ее очень важной численной характеристикой. Вычисление детерминанта в NumPy возможно с помощью функции `np.linalg.det`. Однако, поскольку вычисление детерминанта связано с вычислениями произведений, в тех случаях, когда матрица является очень большой, а её элементы - это очень большие числа, в функции `np.linalg.det` может произойти переполнение, и посчитанное значение нельзя будет использовать в дальнейших вычислениях. С этой целью в NumPy реализована специальная функция `np.linalg.slogdet`, которая возвращает знак детерминанта и натуральный логарифм его модуля.

In [22]:
matrix = np.diag(np.random.randint(1, 5, size=4))

print(
    f"matrix:\n{matrix}",
    f"det:\n{np.linalg.det(matrix)}",
    f"slogdet:\n{np.linalg.slogdet(matrix)}",
    sep="\n\n",
)

matrix:
[[4 0 0 0]
 [0 2 0 0]
 [0 0 3 0]
 [0 0 0 1]]

det:
23.999999999999993

slogdet:
SlogdetResult(sign=1.0, logabsdet=3.1780538303479453)


**Собственные числа**

Собственные числа и собственные векторы матрицы также являются очень важным ее характеристиками. Их вычисление может быть полезно, как для общего анализа некоторого линейного оператора, так и в качества важного шага в каком-либо алгоритме. Например, вычисление собственных чисел является важным шагом при выполнении алгоритма поиска особых точек изображения. Вычисление собственных чисел реализуется с помощью функции `numpy.linalg.eig`.

In [23]:
matrix = np.diag(np.random.randint(1, 5, size=4))
eigen_values, eigen_vectors = np.linalg.eig(matrix)

print(
    f"matrix:\n{matrix}",
    f"eigen values:\n{eigen_values}",
    f"eigen vectors:\n{eigen_vectors}",
    sep="\n\n",
)

matrix:
[[4 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 3]]

eigen values:
[4. 1. 1. 3.]

eigen vectors:
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


## Системы уравнений

Разумеется, одной из важных частей линейной алгебры являются линейные уравнения. Поэтому часть функционала подмодуля `linalg` посвящена решению именно этой проблемы. В этом блоке мы рассмотрим основные функции, которые могут оказаться полезными при решении практических задач.

**Обращение матрицы**

При решении многих задач возникает необходимость в вычислении обратной матрицы. Такая задача, например, может возникать при вычислении коэффициентов МНК, о чем мы поговорим ниже. В NumPy вычисление обратной матрицы возможно с помощью специальной функции `np.linalg.inv`. 

In [24]:
matrix = np.random.randint(-10, 10, size=(3, 3))
matrix_inv = np.linalg.inv(matrix)

print(
    f"matrix:\n{matrix}",
    f"matrix inverse:\n{matrix_inv}",
    f"product:\n{np.round(matrix_inv @ matrix, 2)}",
    sep="\n\n",
)

matrix:
[[-9  1 -5]
 [-7  0  6]
 [-5 -6  9]]

matrix inverse:
[[-0.07185629 -0.04191617 -0.01197605]
 [-0.06586826  0.21157685 -0.17764471]
 [-0.08383234  0.11776447 -0.01397206]]

product:
[[ 1.  0. -0.]
 [-0.  1.  0.]
 [-0.  0.  1.]]


**Решение системы линейных уравнений**

В части случаев имеется возможность получения точного аналитического решения системы линейных уравнений. В этих случаях система линейных уравнений может быть решена с помощью специальной функции `np.linalg.solve`. Аргументами функции являются матрица коэффициентов и столбец правой части.

Для примера рассмотрим решение системы линейных уравнений:

$$
\begin{equation*}
 \begin{cases}
   x_1 + 2x_2 = 1 \\
   3x_1 + 5x_2 = 2
 \end{cases}
\end{equation*}
$$

In [25]:
equation_matrix = np.array([[1, 2], [3, 5]])
right_part = np.array([1, 2])

solution = np.linalg.solve(equation_matrix, right_part)
print(
    ", ".join([
        f"x{i + 1} = {np.round(solution[i], 2)}"
        for i in range(solution.size)
    ])
)

x1 = -1.0, x2 = 1.0


**МНК**

Стоит отметить, что далеко не всегда существует возможность нахождения точного решения системы линейных уравнений. В реальной задаче может существовать большое количество уравнений, противоречащих друг другу. В таких условиях ищется оптимальное решение системы, в смысле минимизации некоторой ошибки. Собственно, такая постановка задачи очень напоминает нам постановку задачи МНК. На самом деле МНК можно использовать в качестве метода вычисления оптимального решения системы линейных уравнений, а потому этот алгоритм также реализован в NumPy.

В примере ниже мы рассмотрим функционал NumPy для решения классической задачи, в которой используется МНК - восстановление линейной зависимости.

Предположим, что у нас есть несколько тестовых точек, и мы пытаемся по ним восстановить исходную зависимость вида:
$$y = ax + b, $$

где $a$ и $b$ - искомые коэффициенты.

Пусть известны следующие координаты точек:

In [27]:
abscissa = np.array([0, 1, 2, 3])
ordinates = np.array([-1, 0.2, 0.9, 2.1])

Для использования функции `np.linalg.lstsq` необходимо составить матрицу коэффициентов системы линейных уравнений. Т.е. мы должны свести задачу к решению системы линейных уравнений относительно переменных $a$ и $b$. Обладая указанными выше точками запишем следующую систему:

$$
\begin{equation*}
 \begin{cases}
   ax_1 + b = y_1 \Rightarrow b = -1 \\
   ax_2 + b = y_2 \Rightarrow a + b = 0.2 \\
   ax_3 + b = y_3 \Rightarrow 2a + b = 0.9\\
   ax_4 + b = y_4 \Rightarrow 3a + b = 2.1
 \end{cases}
\end{equation*}
$$

Итак, займемся построением матрицы системы.

In [28]:
equation_matrix = np.vstack((abscissa, np.ones_like(abscissa))).T
print(f"exiation_matrix:\n{equation_matrix}", end="\n\n")

coefficients = np.linalg.lstsq(
    equation_matrix, ordinates, rcond=None
)[0]

print(
    f"incline: {np.round(coefficients[0], 2)}",
    f"shift: {np.round(coefficients[1], 2)}",
    sep="\n",
)

exiation_matrix:
[[0 1]
 [1 1]
 [2 1]
 [3 1]]

incline: 1.0
shift: -0.95
